# How to on Spark: Cross Validation
> Run TimeGPT distributedly on top of Spark.

`TimeGPT` works on top of Spark, Dask, and Ray through Fugue. `TimeGPT` will read the input DataFrame and use the corresponding engine. For example, if the input is a Spark DataFrame, StatsForecast will use the existing Spark session to run the forecast.


In [ ]:
#| hide
from nixtlats.utils import colab_badge

In [ ]:
#| echo: false
colab_badge('docs/how-to-guides/1_distributed_cv_spark')

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nixtla/nixtla/blob/main/nbs/docs/how-to-guides/1_distributed_cv_spark.ipynb)

# Installation 

As long as Spark is installed and configured, `TimeGPT` will be able to use it. If executing on a distributed Spark cluster, make use the `nixtlats` library is installed across all the workers.

## Executing on Spark

To run the forecasts distributed on Spark, just pass in a Spark DataFrame instead. 

In [ ]:
#| hide
import os

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

Instantiate `TimeGPT` class.

In [ ]:
from nixtlats import TimeGPT

In [ ]:
timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = 'my_token_provided_by_nixtla'
)

In [ ]:
#| hide
timegpt = TimeGPT()

Use Spark as an engine.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/01 03:35:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Cross validation

In [ ]:
url_df = 'https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short.csv'
spark_df = spark.createDataFrame(pd.read_csv(url_df))
spark_df.show(5)

+---------+-------------------+-----+
|unique_id|                 ds|    y|
+---------+-------------------+-----+
|       BE|2016-12-01 00:00:00| 72.0|
|       BE|2016-12-01 01:00:00| 65.8|
|       BE|2016-12-01 02:00:00|59.99|
|       BE|2016-12-01 03:00:00|50.69|
|       BE|2016-12-01 04:00:00|52.58|
+---------+-------------------+-----+
only showing top 5 rows



In [ ]:
fcst_df = timegpt.cross_validation(spark_df, h=12, n_windows=5, step_size=2)
fcst_df.show(5)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.

+---------+-------------------+-------------------+------------------+
|unique_id|                 ds|             cutoff|           TimeGPT|
+---------+-------------------+-------------------+------------------+
|       FR|2016-12-30 04:00:00|2016-12-30 03:00:00| 44.89374542236328|
|       FR|2016-12-30 05:00:00|2016-12-30 03:00:00| 46.05792999267578|
|       FR|2016-12-30 06:00:00|2016-12-30 03:00:00|48.790077209472656|
|       FR|2016-12-30 07:00:00|2016-12-30 03:00:00| 54.39702606201172|
|       FR|2016-12-30 08:00:00|2016-12-30 03:00:00| 57.59300231933594|
+---------+-------------------+-------------------+------------------+
only showing top 5 rows



INFO:nixtlats.timegpt:Validating inputs...


### Cross validation with exogenous variables

Exogenous variables or external factors are crucial in time series forecasting as they provide additional information that might influence the prediction. These variables could include holiday markers, marketing spending, weather data, or any other external data that correlate with the time series data you are forecasting.

For example, if you're forecasting ice cream sales, temperature data could serve as a useful exogenous variable. On hotter days, ice cream sales may increase.

To incorporate exogenous variables in TimeGPT, you'll need to pair each point in your time series data with the corresponding external data.

Let's see an example.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-with-ex-vars.csv')
spark_df = spark.createDataFrame(df)
spark_df.show(5)

+---------+-------------------+-----+----------+----------+-----+-----+-----+-----+-----+-----+-----+
|unique_id|                 ds|    y|Exogenous1|Exogenous2|day_0|day_1|day_2|day_3|day_4|day_5|day_6|
+---------+-------------------+-----+----------+----------+-----+-----+-----+-----+-----+-----+-----+
|       BE|2016-12-01 00:00:00| 72.0|   61507.0|   71066.0|  0.0|  0.0|  0.0|  1.0|  0.0|  0.0|  0.0|
|       BE|2016-12-01 01:00:00| 65.8|   59528.0|   67311.0|  0.0|  0.0|  0.0|  1.0|  0.0|  0.0|  0.0|
|       BE|2016-12-01 02:00:00|59.99|   58812.0|   67470.0|  0.0|  0.0|  0.0|  1.0|  0.0|  0.0|  0.0|
|       BE|2016-12-01 03:00:00|50.69|   57676.0|   64529.0|  0.0|  0.0|  0.0|  1.0|  0.0|  0.0|  0.0|
|       BE|2016-12-01 04:00:00|52.58|   56804.0|   62773.0|  0.0|  0.0|  0.0|  1.0|  0.0|  0.0|  0.0|
+---------+-------------------+-----+----------+----------+-----+-----+-----+-----+-----+-----+-----+
only showing top 5 rows



Let's call the `cross_validation` method, adding this information:

In [ ]:
timegpt_cv_ex_vars_df = timegpt.cross_validation(
    df=spark_df,
    h=48, 
    level=[80, 90],
    n_windows=5,
)
timegpt_cv_ex_vars_df.show(5)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Using the following exogenous variables: Exogenous1, Exogenous2, day_0, day_1, day_2, day_3, day_4, day_5, day_6
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Using the following exogenous variables: Exogenous1, Exogenous2, day_0, day_1, day_2, day_3, day_4, day_5, day_6
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: H
INFO:nixtlats.timegpt:Using the following exogenous variables: E

+---------+-------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+
|unique_id|                 ds|             cutoff|           TimeGPT|     TimeGPT-lo-90|    TimeGPT-lo-80|     TimeGPT-hi-80|     TimeGPT-hi-90|
+---------+-------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+
|       FR|2016-12-21 00:00:00|2016-12-20 23:00:00| 66.39748296460945| 62.03776876172859|63.28946471509773| 69.50550121412117|  70.7571971674903|
|       FR|2016-12-21 01:00:00|2016-12-20 23:00:00| 63.71841894125738|59.770956050632385|61.16832944845953| 66.26850843405524| 67.66588183188237|
|       FR|2016-12-21 02:00:00|2016-12-20 23:00:00| 61.13784444132001| 58.88184931650312| 59.5156742600456|62.760014622594426|63.393839566136904|
|       FR|2016-12-21 03:00:00|2016-12-20 23:00:00| 55.77490648975175|53.047358607671676|53.22071413745683|58.32909884204667

INFO:nixtlats.timegpt:Validating inputs...


In [ ]:
spark.stop()